In [1]:
# Import things that are needed generically
from langchain.agents import AgentType, initialize_agent
from langchain_cohere import ChatCohere
from langchain.tools import BaseTool
from pydantic import BaseModel
from typing import Optional
from typing import Any, ClassVar


from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

import wandb
import os

import os
import random
import time
import datetime

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/michieldekoninck/Library/Application Support/sagemaker/config.yaml


In [2]:
wandb.init(project='worldpicker', job_type="generation")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: michieldekoninck2 (michieldekoninck2-mdk). Use `wandb login --relogin` to force relogin


In [3]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

In [4]:
!echo $LANGCHAIN_WANDB_TRACING

true


In [5]:
class WorldPickerTool(BaseTool):
    name : str  = "pick_world"
    description : str = "pick a virtual game world for your character or item naming"
    worlds : ClassVar[list['str']] = [
                "a mystic medieval island inhabited by intelligent and funny frogs",
                "a modern anthill featuring a cyber-ant queen and her cyber-ant-workers",
                "a digital world inhabited by friendly machine learning engineers"
            ]

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        time.sleep(1)
        return random.choice(self.worlds)

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("pick_world does not support async")
        
class NameValidatorTool(BaseTool):
    name :str = "validate_name"
    description :str = "validate if the name is properly generated"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        time.sleep(1)
        if len(query) < 20:
            return f"This is a correct name: {query}"
        else:
            return f"This name is too long. It should be shorter than 20 characters."

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("validate_name does not support async")

In [6]:
from dotenv import load_dotenv
import os

load_dotenv()
COHERE_API_KEY= os.getenv("COHERE_API_KEY")


In [7]:
llm = ChatCohere(cohere_api_key=COHERE_API_KEY, model='command-r')

In [8]:
llm

ChatCohere(client=<cohere.client.Client object at 0x105447b20>, async_client=<cohere.client.AsyncClient object at 0x28280c3d0>, model='command-r', cohere_api_key=SecretStr('**********'))

In [9]:
tools = [WorldPickerTool(), NameValidatorTool()]
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_42614/2004885880.py:2: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [10]:
from langchain.callbacks import wandb_tracing_enabled
with wandb_tracing_enabled():
    agent.run(
        "Find a virtual game world for me and imagine the name of a hero in that world"
    )

/var/folders/3l/70h6qcrd72q0j5x4sscw7qn40000gn/T/ipykernel_42614/223273874.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run(




> Entering new AgentExecutor chain...
Action: pick_world
Action Input: None
Observation:
Observation: a mystic medieval island inhabited by intelligent and funny frogs
Thought:Action: validate_name
Action Input: Frogy the Hero
Observation:
Observation: This name is too long. It should be shorter than 20 characters.
Thought:Question: What's a cool name for a hero in a world inhabited by funny frogs?
Thought: I should pick a shorter name.
Action: pick_world
Action Input: None
Observation:
Observation: a digital world inhabited by friendly machine learning engineers
Thought:Thought: I'll keep the same virtual game world, as it seems suitable for a fun hero name.
Action: validate_name
Action Input: Hero Froggy
Observation:
Observation: This name is too long. It should be shorter than 20 characters.
Thought:Question: What's a suitable name for a hero in a world inhabited by friendly machine learning engineers?

Thought: It's best to keep the name short, snappy, and fun!

Action: pick_worl

wandb: ERROR WARNING: Failed to serialize model: 'NoneType' object has no attribute 'items'


Final Answer: The name 'Froggy' would suit a hero in a world inhabited by funny frogs. It's short, snappy, and easy to remember!

> Finished chain.


In [11]:
wandb.finish()